In [ ]:
import h5py
import matplotlib.pyplot as plt
import os
from os import listdir
from mpl_toolkits import mplot3d
import matplotlib.gridspec as gridspec
import numpy as np
import re

axisScale = 0.03
datasets = ['organic', 'gm_late', 'gm_early']

datasetNo = 0

# define storage arrays and reshape to hold all three galaxy type data
redshifts = np.array(range(3*24), dtype=float)
redshifts.shape = (3, 24)
kco_s = np.array(range(3*24), dtype=float)
kco_s.shape = (3, 24)

# Separate each star particle in each data set with '---'
for dataset in datasets:
    print('-----------------------------------------')
    print(dataset)
    print('-----------------------------------------')
    
    #Where to find the files
    files = listdir('C:\\Users\\katha\\GalaxyMorphology\\' + dataset)
    os.path.join("/C:", "Users","katha","GalaxyMorphology", dataset)

    #Start counting from 0
    count = 0

    #Reshape angular momentum - DO WE NEED THIS???
    #galaxyAm = np.array(range(96), dtype=float).reshape(24,4)
    
    for file in files:
    #if 1==1:


    
        #Example file name (EXPLAIN FILE NAME STRUCTURE)
        #file = 'star_particles_015_z002p012.hdf5'
        #file = 'star_particles_028_z000p000.hdf5'
        
        # get redshift from the filename
        m = re.search('(z[0-9])\w+', file)
        s = m.group(0).replace('z', '')
        s = s.replace('p', '.')
        redshift = float(s)
        
        # load data for a particular galaxy at a particular redshift
        f = h5py.File('C:\\Users\\katha\\GalaxyMorphology\\' + dataset + '/' + file,'r')

        # extract coordinate, velocity and mass data from the file
        ds_c = f['Coordinates']
        ds_v = f['Velocity']
        ds_m = f['Mass']
        
         # Calculate the resultant angular momentum vectors j
        # j contains the AM vectors for each particle per redshift era
        j = np.cross(ds_c, ds_v)
        specAngMom = np.transpose(np.multiply(ds_m, np.transpose(j)))
        specAngMomTot = np.sum(specAngMom, axis=0)/(np.sum(ds_m))
        specAngMomTotMag = np.linalg.norm(specAngMomTot)
        #print(np.shape(j))
        #print(np.shape(specAngMom))
        #print(np.shape(specAngMomTot))
        #print(np.shape(specAngMomTotMag))
        
       
        # Calculate the z unit vector
        unitVect_z = specAngMomTot / specAngMomTotMag
        #print (unitVect_z)
        #print (np.linalg.norm(unitVect_z))
        
        # Calculate x and y
        
        # the angular momentum's vector's (unitVect_z) direction is directly out of the plane of the galaxy
        # unitVect_z = k, but j = [-k2/k1, 1, 0], so
        
        k = unitVect_z
        print ("k: ", k)
        
        j_ = [-k[1]/k[0], 1, 0]
        j_=j_/(np.linalg.norm(j_))
        print ("j_: ", j_)
        #print (np.linalg.norm(j_))
        
        i = np.cross(j_, unitVect_z)
        i=i/(np.linalg.norm(i))
        print ("i: ", i)
        #print (np.linalg.norm(i))
        
        dsc_x_trsfrm = np.dot(ds_c, i)
        #print(ds_c[0])
        #print ("dsc_x_trsfrm", dsc_x_trsfrm)
        #print ("dsc_x_trsfrm shape", np.shape(dsc_x_trsfrm))
        
        dsc_y_trsfrm = np.dot(ds_c, j_)
        #print ("dsc_y_trsfrm", dsc_y_trsfrm)
        #print ("dsc_y_trsfrm shape", np.shape(dsc_y_trsfrm))
        
        dsc_z_trsfrm = np.dot(ds_c, k)
        #print ("dsc_z_trsfrm", dsc_z_trsfrm)
        #print ("dsc_z_trsfrm shape", np.shape(dsc_z_trsfrm))
        
         #dsc_trans = list(zip(dsc_x_trsfrm, dsc_y_trsfrm, dsc_z_trsfrm))
        dsc_trans = np.transpose(np.array([dsc_x_trsfrm, dsc_y_trsfrm, dsc_z_trsfrm]))
                
          #transform velocities
        dsv_x_trsfrm = np.dot(ds_v, i)
        #print ("dsv_x_trsfrm", dsv_x_trsfrm)
        #print ("dsv_x_trsfrm shape", np.shape(dsv_x_trsfrm))
        
        dsv_y_trsfrm = np.dot(ds_v, j_)
        #print ("dsv_y_trsfrm", dsv_y_trsfrm)
        #print ("dsv_y_trsfrm shape", np.shape(dsv_y_trsfrm))
        
        dsv_z_trsfrm = np.dot(ds_v, k)
        #print ("dsv_z_trsfrm", dsv_z_trsfrm)
        #print ("dsv_z_trsfrm shape", np.shape(dsv_z_trsfrm))
        
        dsv_trans = np.transpose(np.array([dsv_x_trsfrm, dsv_y_trsfrm, dsv_z_trsfrm]))
        
        # Calculate KE of transformed particles
        # Get magnitudes of the vectors
        vel_magnitude = np.linalg.norm(dsv_trans, axis=1)

        # Calculate kinetic energy for all star particles
        K_tot = np.sum(0.5 * np.array(ds_m) * np.square(vel_magnitude))
        print ('K_tot: ', K_tot)
        ##specificKe = np.sum(0.5 * np.square(vel_magnitude))

        # Calculate R, the distance from the centre in the x-y plane (R = sqrt(x^2 + y^2))
        R = np.sqrt(np.square(dsc_x_trsfrm) + np.square(dsc_y_trsfrm))
        #print to test shapes
        #print ('R shape', np.shape(R))
        #print ('R:', R)
        
        # Compute the particle’s angular momentum vector (L = m_star*(v cross r))
        res = np.cross(dsc_trans, dsv_trans)
        momentum = np.transpose(np.multiply(ds_m, np.transpose(res)))
         ##momentum = np.multiply(ds_m, res)
        #print('Momentum shape:', np.shape(momentum))
        #print('Momentum',momentum)
        
        #  L_z is the 3rd component (the z component) of the resulting vector
        L_z = np.array(momentum[:,2])
        #print('L_z:', L_z)  
        
        
         #The numerator is then the sum of 0.5*m_star* [L_z / (m_star * R)]^2, but *only* considering 
         #those particles for which L_z is *positive*. If L_z is negative, it means that the particle is 
        #counter-rotating, and we don’t want to treat counter-rotation as co-rotation
        
        #Ignore the loop and continue if L_z is less than zero so we only consider positive values
        #Ignore cases where R=o as it leads to inifinities. There should be no more than 1 per galaxy, per redshift 
        #– the centre of potential is chosen to be the particle with the lowest potential, so finding 1 such case 
        # is not a problem, but you shouldn’t find more than 1.
        r0 = 0
        Krot_co = 0
        for n in range(0, len(ds_m)):
            if L_z[n] < 0:
                continue
            if R[n] == 0:
                r0 = r0 + 1
                continue
            Krot_co = Krot_co + (0.5 * ds_m[n] * np.square(L_z[n] / (ds_m[n]*R[n]) ))
            
        # Show how many R=0 cases there are
        print('R=0 count:',r0)
        
        print('Krot_co',Krot_co)
        
        #kappa_co is then the numerator divided by the denominator. It should be a number between 0 and 1!
        K_co = Krot_co / K_tot
        
        print('K_co',K_co)
        
        redshifts[datasetNo,count] = redshift
        kco_s[datasetNo,count] = K_co
              
        count = count + 1
        print ('-------------------------------------------------')
         
    # Next galaxy
    datasetNo = datasetNo + 1

-----------------------------------------
organic
-----------------------------------------
k:  [0.9515281  0.30598536 0.0311003 ]
j_:  [-0.30613344  0.95198861  0.        ]
i:  [ 0.02960713  0.00952084 -0.99951627]
K_tot:  36702494430.6407
R=0 count: 0
Krot_co 5524279878.332274
K_co 0.15051510705278892
-------------------------------------------------
k:  [ 0.65267355 -0.15264495  0.74210293]
j_:  [0.22773109 0.97372406 0.        ]
i:  [ 0.72260348 -0.16899991 -0.67028594]
K_tot:  119774170054.30975
R=0 count: 0
Krot_co 37546370059.415985
K_co 0.3134763533939844
-------------------------------------------------
k:  [ 0.28733915  0.50012508 -0.81689113]
j_:  [-0.86708064  0.49816781  0.        ]
i:  [-0.40694887 -0.70831048 -0.57679189]
K_tot:  160746946946.82263
R=0 count: 0
Krot_co 42588177690.39494
K_co 0.264939263228954
-------------------------------------------------
k:  [-0.0927266   0.99391521  0.05945022]
j_:  [0.99567629 0.0928909  0.        ]
i:  [ 0.00552238 -0.05919317  0.